# based-trees

Ideias:
- FEITO: Usar a coluna para no texto
- FEITO: Limpar os dados de conteudo
- FEITO (lucas): ordenar os itens novos que ficaram por ultimo por popularidade
- FEITO: Adicionar o nome da coluna, porque vai indicar o assunto, em conjunto do texto

- tunar os hiperparametros
- usa a imagem alem do texto
- usar algum algoritmo mais simples apenas para usuarios novos, knn para content based
- Entender o parametro exclude_unknowns=True do RatioSplit e se tem alguma forma do DMRL gerar previsoes para itens e usuarios novos 

- paralelizar a previsao
Perguntas:


## Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import torch
import cornac
import numpy as np
import pandas as pd
from cornac.metrics import NDCG
from cornac.eval_methods import RatioSplit
from cornac.data import TextModality, ImageModality
from cornac.models.dmrl.recom_dmrl import DMRL 
from tqdm import tqdm
from utils import load_data, preprocessing_content_data
import requests
from PIL import Image
from io import BytesIO
from joblib import Parallel, delayed


# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
np.set_printoptions(threshold=np.inf)

## Load and process data

### Load data

In [3]:
ratings, content, targets = load_data()

In [4]:
ratings["TimestampDate"] = ratings['Timestamp'].dt.date
ratings.loc[ratings.Rating == 0, "Rating"] = 0.01

### Limpar os dados de conteudo

- FEITO - Year: deletar o - no texto '2013–'
- FEITO - Rated: alterar as diversas formas de escrever NA para NA. Esse e um caso especial
- FEITO - alterar as diversas formas de escrever NA para None em todas as coluans
- FEITO - Language: tem varias linhas que possuem bizarices como  'None, English' e 'None, French' , 'English, None'...
- FEITO - Ratings: criar uma coluna para cada chave do dicionario, entender quais sao todas as chaves que existem

-----------------------------------------------
Variaveis que nao precisariam ser tratadas com um bert:
- Metascore
- imdbRating
- Type

In [5]:
content_auxiliar = content.drop(columns=["Poster", "Website", "Response", "Episode", "seriesID", "Season"]).copy()

In [6]:
content_auxiliar['Year'] = content_auxiliar['Year'].str.replace('–', '')

In [7]:
nan = content.totalSeasons.unique()[0]
dict_transform_to_na = {
    "Rated":['N/A', 'Not Rated', 'Unrated', 'UNRATED', 'NOT RATED'],
    "all": [nan, 'N/A', 'None', np.nan],
}

for na_value in dict_transform_to_na["all"]:
    content_auxiliar = content_auxiliar.replace(na_value, None)

for na_value in dict_transform_to_na["Rated"]:
    content_auxiliar['Rated'] = content_auxiliar['Rated'].replace(na_value, None)

In [8]:
content_auxiliar['Language'] = content_auxiliar['Language'].str.replace('None, ', '')
content_auxiliar['Language'] = content_auxiliar['Language'].str.replace(', None', '')

In [9]:
# Entendendo os valores possiveis para a coluna Ratings
# A coluna content_auxiliar.Ratings quarda uma lista que posde ter entre 0 e 3 dicionarios. Cada dicionario possui a chave 'Source', 'Value'.
num_ratings_per_item = []
unique_keys = []
rating_sources = []
rating_values = []

for rating_list in content_auxiliar.Ratings:
    num_ratings_per_item.append(len(rating_list))
    for rating_dict in rating_list:
        for key in rating_dict:
            unique_keys.append(key)
        rating_sources.append(rating_dict['Source'])
        rating_values.append(rating_dict['Value'])

set(rating_sources)

{'Internet Movie Database', 'Metacritic', 'Rotten Tomatoes'}

In [10]:
InternetMovieDatabase_list = []
Metacritic_list = []
RottenTomatoes_list = []
for rating_list in content_auxiliar.Ratings:
    InternetMovieDatabase_list.append(None)
    Metacritic_list.append(None)
    RottenTomatoes_list.append(None)
    for rating_dict in rating_list:
        if rating_dict['Source'] == 'Internet Movie Database':
            InternetMovieDatabase_list[-1] = rating_dict['Value']
        elif rating_dict['Source'] == 'Metacritic':
            Metacritic_list[-1] = rating_dict['Value']
        elif rating_dict['Source'] == 'Rotten Tomatoes':
            RottenTomatoes_list[-1] = rating_dict['Value']

In [11]:
content_auxiliar['Internet Movie Database'] = InternetMovieDatabase_list
content_auxiliar['Metacritic'] = Metacritic_list
content_auxiliar['Rotten Tomatoes'] = RottenTomatoes_list


In [12]:
content_auxiliar.drop(columns=['Ratings'], inplace=True)

### Apendar a coluna no valor do dataframe

In [13]:
# content_columns = content_auxiliar.columns.to_list()
# content_columns.pop(0)

In [14]:
# for column in content_columns:
#     content_auxiliar[column] = content_auxiliar[column].apply(lambda x: f"{column}: {x}; " if x is not None else f"{column}: unknown value; ")

In [15]:
# content_processed = content_auxiliar[['ItemId']].copy()
# content_processed["text"] = content_auxiliar[content_columns].astype(str).fillna('').agg(' '.join, axis=1)

### Viabiliza as colunas para um modelo tabular

In [16]:
content_auxiliar["Internet Movie Database"] = content_auxiliar["Internet Movie Database"].str.split("/").apply(lambda x: x[0] if x is not None else None)
content_auxiliar["Rotten Tomatoes"].replace('%', "", regex=True, inplace=True)
content_auxiliar["Metacritic"].replace('/100', "", regex=True, inplace=True)
content_auxiliar["imdbVotes"].replace(',', "", regex=True, inplace=True)

In [17]:
content_auxiliar["BoxOffice"].replace(',', "", regex=True, inplace=True)

# O metodo replace nao funcionnou para $, cuidado para nao rodar essa cedula 2 vezes
content_auxiliar["BoxOffice"] = content_auxiliar["BoxOffice"].apply(lambda x: x[1:] if x is not None else None) # troca valores $ por ""


In [18]:
content_auxiliar["Runtime"].replace(' min', "", regex=True, inplace=True)
mask = content_auxiliar.Runtime.str.contains('h', na=False)
content_auxiliar.loc[mask, "Runtime"] = content_auxiliar.loc[mask, "Runtime"].str.split("h").apply(lambda x: str(int(x[0]) * 60 + int(x[1])))

In [19]:
columns_to_float = ['Runtime', 'Internet Movie Database', 'Metacritic',
                    'Rotten Tomatoes', "imdbVotes", "BoxOffice",
                    "totalSeasons", "imdbRating", "Metascore"]  # Falta tartar Awards
content_auxiliar[columns_to_float] = content_auxiliar[columns_to_float].astype(float)

In [20]:
variaveis_textos = ["Title", "Plot"]
variaveis_categoricas = ["Rated", "Director", "Production"] # variaveis para tratar "Language", Writer, Actors, Country
variaveis_data = ["Released", "Year", "DVD"] # Todas faltam tratar 

In [21]:
content_auxiliar.drop(columns=columns_to_float+variaveis_textos+variaveis_categoricas).head(2)

,ItemId,Year,Released,Genre,Writer,Actors,Language,Country,Awards,Type,DVD
0,c9f0f895fb,1894,09 Jan 1894,"Documentary, Short",None,Fred Ott,None,United States,1 win,movie,None
1,d3d9446802,1895,22 Mar 1895,"Documentary, Short",None,None,None,France,None,movie,None


In [22]:
content_auxiliar["Production"].nunique()

458

In [23]:
content_auxiliar.shape

(38012, 25)

In [24]:
# transforma as colunas em numericas: Runtime

# usar o catboost para categoria e para suportar NA...
# transformar em categorias... Genre, Director, Writer, Actors, Language, Country, Type, Production

# Year transformar em anos ate 2025
# Released, DVD transformar em dias ate 2025

## Train model

In [25]:
from catboost import Pool, CatBoostRegressor

In [26]:
content_auxiliar[columns_to_float] = content_auxiliar[columns_to_float].fillna(-1)
content_auxiliar[variaveis_categoricas] = content_auxiliar[variaveis_categoricas].fillna("unknown")

In [27]:
train_data = ratings[["UserId", "ItemId", "Rating"]].merge(content_auxiliar[["ItemId"] + columns_to_float + variaveis_categoricas], on="ItemId")

In [30]:
target = "Rating"

In [29]:
train_data.set_index(["UserId", "ItemId"], inplace=True)

In [31]:
model = CatBoostRegressor(
    iterations=1000,
    task_type="GPU",    
    devices='0',
    cat_features=variaveis_categoricas

)

In [32]:
model.fit(train_data.drop(columns=target),
          train_data[target],
          verbose=True)

Learning rate set to 0.094547
0:	learn: 1.8238561	total: 43.3ms	remaining: 43.2s
1:	learn: 1.7923972	total: 82.5ms	remaining: 41.2s
2:	learn: 1.7660901	total: 116ms	remaining: 38.4s
3:	learn: 1.7439051	total: 147ms	remaining: 36.7s
4:	learn: 1.7251866	total: 182ms	remaining: 36.2s
5:	learn: 1.7096156	total: 209ms	remaining: 34.7s
6:	learn: 1.6967982	total: 253ms	remaining: 35.9s
7:	learn: 1.6859841	total: 299ms	remaining: 37s
8:	learn: 1.6768324	total: 329ms	remaining: 36.2s
9:	learn: 1.6688609	total: 360ms	remaining: 35.6s
10:	learn: 1.6621517	total: 391ms	remaining: 35.2s
11:	learn: 1.6566992	total: 420ms	remaining: 34.5s
12:	learn: 1.6520459	total: 447ms	remaining: 33.9s
13:	learn: 1.6481457	total: 485ms	remaining: 34.1s
14:	learn: 1.6448184	total: 513ms	remaining: 33.7s
15:	learn: 1.6419763	total: 542ms	remaining: 33.3s
16:	learn: 1.6395073	total: 570ms	remaining: 33s
17:	learn: 1.6374935	total: 597ms	remaining: 32.5s
18:	learn: 1.6358165	total: 626ms	remaining: 32.3s
19:	learn: 1.

### Preve para os dados de teste

In [ ]:
test_data = targets.merge(content_auxiliar[["ItemId"] + columns_to_float + variaveis_categoricas], on="ItemId")
test_data.set_index(["UserId", "ItemId"], inplace=True)

In [44]:
previsao_test = model.predict(test_data)

In [51]:
target_prediction = test_data.reset_index()[["UserId", "ItemId"]]

In [52]:
target_prediction["Rating"] = previsao_test

In [53]:
target_prediction = target_prediction.sort_values(["UserId", "Rating"], ascending=[True, False])

In [54]:
target_prediction.to_csv("submissoes/catboost_versao_1.csv", index=False)

In [55]:
target_prediction = target_prediction.drop(columns="Rating")

In [56]:
target_prediction.to_csv("submissoes/catboost_versao_1_sem_rating.csv", index=False)